# EAA Summe School in Computational Acoustics 2021


## Numerical integration

The objective of this first tutorial is to present numerical integration used for the computation of the element matrices

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import legendre

In [2]:
%matplotlib inline
import matplotlib as mpl
mpl.rc('lines', linewidth=2)
mpl.rc('font', size=14)
mpl.rc('axes', linewidth=1.5, labelsize=14)
mpl.rc('legend', fontsize=14)

### Definition of the problem

The expression for the $(i,j)$ entries of the mass and stiffness matrices are:
$$[\boldsymbol{M}_e]_{i,j}=
\displaystyle{\int_0^\delta \phi_i(x)\phi_j(x) dx},\quad
[\boldsymbol{K}_e]_{i,j}=
\displaystyle{\int_0^\delta \phi_i'(x)\phi_j'(x) dx}.$$

The $\phi$ functions are polynomials that relates the values of the fields in terms of the degrees of freedom. These expressions depend on the length $\delta$ of the element. Instead of integrating between $0$ and $\delta$, one solution is to convert the integration domain to a (standardised) reference interval ]-1;1[:

<div class="alert alert-block alert-info">
Show that 
$$ \displaystyle{\int_0^\delta \phi_i(x)\phi_j(x) dx}=\dfrac{\delta}{2}\displaystyle{\int_{-1}^1 N_i(\xi)N_j(\xi) d\xi},\quad \displaystyle{\int_0^\delta \phi_i'(x)\phi_j'(x) dx}=\dfrac{2}{\delta}\displaystyle{\int_{-1}^1 N'_i(\xi)N'_j(\xi) d\xi}$$
and what are the expression of the shape functions $N$?

As an example, for linear elements, the two $\phi$ functions are:
$$ \phi_1(x)=1-\dfrac{x}{\delta},\quad \phi_2(x)=\dfrac{x}{\delta}.$$

<div class="alert alert-block alert-info">
What are the corresponding expressions of $N_1(\xi)$ and $N_2(\xi)$? Compute the integrals $\displaystyle{\int_{-1}^1 N_i'(\xi)N_j'(\xi) d\xi}$ and show that the expression of the stiffness matrix is:
$$ [\boldsymbol{K}_e] =\dfrac{1}{\delta}\begin{bmatrix}
1 & -1\\-1 & 1
\end{bmatrix}.$$
To obtain these four coefficients, we then need to compute four integrals, but this number can be reduced. How many independent integrations are necessary? 

We can compute the mass matrix in a similar way:

<div class="alert alert-block alert-info">
Show that the expression of the mass matrix is:
$$ [\boldsymbol{M}_e] =\dfrac{\delta}{6}\begin{bmatrix}
2 & 1\\1 & 2
\end{bmatrix}.$$

To obtain the previous expressions, we have first computed the product of the two shape functions (or their derivatives) then integrated this product. It was relatively simple as the degrees for linear elements were rather low but this can become time consuming for higher-order shape functions or when the elements will be two or three dimensional. Another approach consists in using a numerical integration.

All the schemes share the same formalism and the integral is approximated as follows:
$$ \displaystyle{\int_{-1}^1} f(\xi) d\xi \approx \displaystyle{\sum_{k=0}^n} w_kf(x_k).$$

The integral is thus replaced by a linear combination of the values of the function at some particular points $x_k$ called nodes.
A weight $w_k$ is associated to each node.
The different schemes will differ by the number $n+1$ of points, their location, and the values of the weights. 

A classical integration rule is the Simpson rule for which the coefficients are:
$$ x_0=-1,\quad x_1=0,\quad x_2=1,\quad
w_0=\dfrac{1}{6},\quad w_1=\dfrac{4}{6},\quad w_2=\dfrac{1}{6}.$$

<div class="alert alert-block alert-info">
Apply the rule to the mass matrix (for which the polynomial to integrate is of degree 2).
What is the quality of the approximation?

It can be shown rigorously that this scheme provides the exact solution for any polynomial of degree lower than 2. The Simpson quadrature method is then called second order. We can show that with a set of $n$ equidistant points, we can choose the coefficients so that the method is of order $n-1$.

We can therefore replace the formal integration by a linear combination of values of the function.
But we can do much better, let's try the following set of points and coefficients:
$$ x_0=-\sqrt{1/3},\quad x_1=\sqrt{1/3},\quad
w_0=1,\quad w_1=1.$$

<div class="alert alert-block alert-info">
Show that for every polynomial $P(X)=aX^3+bX^2+cX+d$ of degree lower than 3, we have:
$$ \displaystyle{\int_{-1}^1} P(\xi) d\xi = w_0P(x_0)+w_1P(x_1).$$ 

With just 2 points, we are able to integrate exactly a polynomial of degree 3 ! 

This can be generalised.
With Gauss-Legendre integration we can integrate exactly a polynomial of degree $2n+1$ with $n+1$ points.
The points $x_k$ are the roots of the Legendre polynomial $L_n$ of degree $n$ and the weights $w_k$ are given by:
$$ w_k=\dfrac{2}{(1-x_k^2)L_n'(x_k)^2}.$$

The quadrature rule is then implemented as follows:

In [3]:
n = 3
def Gauss_Legendre_coefficients(n):
    L_n = legendre(n) # Legendre polynomial of degree n 
    x = np.roots(L_n) # Gauss points are the roots of L_n
    dL_n = np.polyder(L_n) # derivative of L_n
    w = 2/((1-x**2)*np.polyval(dL_n,x)**2) # weights
    return x,w

def Gauss_Legendre_quad(P,x,w):
    sum = 0
    for k in range(len(x)):
        sum += w[k]*np.polyval(P,x[k])
    return sum

We can then apply this method to compute the mass matrix for linear elements. Since the integrands are quadratic, only 2 Gauss points are necessary

In [4]:
n = 2
x,w = Gauss_Legendre_coefficients(n)
# Vector of the shape functions
N = []
N.append(np.poly1d([-1/2,1/2]))
N.append(np.poly1d([1/2,1/2]))
M = np.zeros((2,2))
for i in range(2):
    for j in range(i,2):
        M[i,j] = Gauss_Legendre_quad(N[i]*N[j],x,w)
        M[j,i] = M[i,j]

print(M)

[[0.66666667 0.33333333]
 [0.33333333 0.66666667]]


<div class="alert alert-block alert-info">
Use this method to compute the mass and stiffness matrices associated to quadratic elements.